In [30]:
VALIDATE = True

MAX_ROUNDS = 500
EARLY_STOP = 50
OPT_ROUNDS = 500

In [1]:
import pandas as pd
import time
import numpy as np
from sklearn.model_selection import train_test_split 
import lightgbm as lgb

In [8]:
path = 'input/'

In [9]:
dtypes = {
        'ip'            : 'uint32',
        'app'           : 'uint16',
        'device'        : 'uint16',
        'os'            : 'uint16',
        'channel'       : 'uint16',
        'is_attributed' : 'uint8',
        'click_id'      : 'uint32'
        }

In [20]:
print('load train...')
train_cols = ['ip','app','device','os', 'channel', 'click_time', 'is_attributed']
train_df = pd.read_csv(path+"train.csv", skiprows=range(1,109903891), nrows=750000,dtype=dtypes, usecols=train_cols)

load train...


In [21]:
import gc

len_train = len(train_df)
print(len_train)
gc.collect()

print('data prep...')

most_freq_hours_in_test_data = [4, 5, 9, 10, 13, 14]
least_freq_hours_in_test_data = [6, 11, 15]


750000


7

data prep...


In [11]:
def prep_data( df ):
    
    df['hour'] = pd.to_datetime(df.click_time).dt.hour.astype('uint8')
    df['day'] = pd.to_datetime(df.click_time).dt.day.astype('uint8')
    df.drop(['click_time'], axis=1, inplace=True)
    gc.collect()
    
    df['in_test_hh'] = (   3 
                         - 2*df['hour'].isin(  most_freq_hours_in_test_data ) 
                         - 1*df['hour'].isin( least_freq_hours_in_test_data ) ).astype('uint8')
    print( df.info() )

#     print('group by : ip_day_test_hh')
#     gp = df[['ip', 'day', 'in_test_hh', 'channel']].groupby(by=['ip', 'day',
#              'in_test_hh'])[['channel']].count().reset_index().rename(index=str, 
#              columns={'channel': 'nip_day_test_hh'})
#     df = df.merge(gp, on=['ip','day','in_test_hh'], how='left')
#     del gp
#     df.drop(['in_test_hh'], axis=1, inplace=True)
#     print( "nip_day_test_hh max value = ", df.nip_day_test_hh.max() )
#     df['nip_day_test_hh'] = df['nip_day_test_hh'].astype('uint32')
#     gc.collect()
#     print( df.info() )

#     print('group by : ip_day_hh')
#     gp = df[['ip', 'day', 'hour', 'channel']].groupby(by=['ip', 'day', 
#              'hour'])[['channel']].count().reset_index().rename(index=str, 
#              columns={'channel': 'nip_day_hh'})
#     df = df.merge(gp, on=['ip','day','hour'], how='left')
#     del gp
#     print( "nip_day_hh max value = ", df.nip_day_hh.max() )
#     df['nip_day_hh'] = df['nip_day_hh'].astype('uint16')
#     gc.collect()
#     print( df.info() )

#     print('group by : ip_hh_os')
#     gp = df[['ip', 'os', 'hour', 'channel']].groupby(by=['ip', 'os', 
#              'hour'])[['channel']].count().reset_index().rename(index=str, 
#              columns={'channel': 'nip_hh_os'})
#     df = df.merge(gp, on=['ip','os','hour'], how='left')
#     del gp
#     print( "nip_hh_os max value = ", df.nip_hh_os.max() )
#     df['nip_hh_os'] = df['nip_hh_os'].astype('uint16')
#     gc.collect()
#     print( df.info() )

#     print('group by : ip_hh_app')
#     gp = df[['ip', 'app', 'hour', 'channel']].groupby(by=['ip', 'app', 
#              'hour'])[['channel']].count().reset_index().rename(index=str, 
#              columns={'channel': 'nip_hh_app'})
#     df = df.merge(gp, on=['ip','app','hour'], how='left')
#     del gp
#     print( "nip_hh_app max value = ", df.nip_hh_app.max() )
#     df['nip_hh_app'] = df['nip_hh_app'].astype('uint16')
#     gc.collect()
#     print( df.info() )

#     print('group by : ip_hh_dev')
#     gp = df[['ip', 'device', 'hour', 'channel']].groupby(by=['ip', 'device', 
#              'hour'])[['channel']].count().reset_index().rename(index=str, 
#              columns={'channel': 'nip_hh_dev'})
#     df = df.merge(gp, on=['ip','device','hour'], how='left')
#     del gp
#     print( "nip_hh_dev max value = ", df.nip_hh_dev.max() )
#     df['nip_hh_dev'] = df['nip_hh_dev'].astype('uint32')
#     gc.collect()
#     print( df.info() )

    df.drop( ['ip','day'], axis=1, inplace=True )
    gc.collect()
    print( df.info() )
    
    return( df )

In [22]:

print( "Train info before: ")
print( train_df.info() )
train_df = prep_data( train_df )
gc.collect()
print( "Train info after: ")
print( train_df.info() )

Train info before: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 750000 entries, 0 to 749999
Data columns (total 7 columns):
ip               750000 non-null uint32
app              750000 non-null uint16
device           750000 non-null uint16
os               750000 non-null uint16
channel          750000 non-null uint16
click_time       750000 non-null object
is_attributed    750000 non-null uint8
dtypes: object(1), uint16(4), uint32(1), uint8(1)
memory usage: 15.0+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 750000 entries, 0 to 749999
Data columns (total 9 columns):
ip               750000 non-null uint32
app              750000 non-null uint16
device           750000 non-null uint16
os               750000 non-null uint16
channel          750000 non-null uint16
is_attributed    750000 non-null uint8
hour             750000 non-null uint8
day              750000 non-null uint8
in_test_hh       750000 non-null uint8
dtypes: uint16(4), uint32(1), uint8(4)
memory u

27

Train info after: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 750000 entries, 0 to 749999
Data columns (total 7 columns):
app              750000 non-null uint16
device           750000 non-null uint16
os               750000 non-null uint16
channel          750000 non-null uint16
is_attributed    750000 non-null uint8
hour             750000 non-null uint8
in_test_hh       750000 non-null uint8
dtypes: uint16(4), uint8(3)
memory usage: 7.9 MB
None


In [23]:
print("vars and data type: ")
train_df.info()

vars and data type: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 750000 entries, 0 to 749999
Data columns (total 7 columns):
app              750000 non-null uint16
device           750000 non-null uint16
os               750000 non-null uint16
channel          750000 non-null uint16
is_attributed    750000 non-null uint8
hour             750000 non-null uint8
in_test_hh       750000 non-null uint8
dtypes: uint16(4), uint8(3)
memory usage: 7.9 MB


In [24]:
metrics = 'auc'

In [25]:
lgb_params = {
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric':metrics,
        'learning_rate': 0.1,
        'num_leaves': 7,  # we should let it be smaller than 2^(max_depth)
        'max_depth': 4,  # -1 means no limit
        'min_child_samples': 100,  # Minimum number of data need in a child(min_data_in_leaf)
        'max_bin': 100,  # Number of bucketed bin for feature values
        'subsample': 0.7,  # Subsample ratio of the training instance.
        'subsample_freq': 1,  # frequence of subsample, <=0 means no enable
        'colsample_bytree': 0.7,  # Subsample ratio of columns when constructing each tree.
        'min_child_weight': 0,  # Minimum sum of instance weight(hessian) needed in a child(leaf)
        'min_split_gain': 0,  # lambda_l1, lambda_l2 and min_gain_to_split to regularization
        'nthread': 4,
        'verbose': 0,
        'scale_pos_weight':99.7, # because training data is extremely unbalanced 
        'metric':metrics
}

In [32]:
target = 'is_attributed'
predictors = ['app','device','os', 'channel', 'hour']#,'nip_day_test_hh', 'nip_day_hh',
#               'nip_hh_os', 'nip_hh_app', 'nip_hh_dev'
categorical = ['app', 'device', 'os', 'channel', 'hour']

In [33]:
print(train_df.head(5))


        app  device  os  channel  is_attributed  hour  in_test_hh
425729   12       1  19      259              0    12           3
447007    9       1  17      334              0    12           3
25761     3       1  13      280              0    12           3
471245   27       1   8      153              0    12           3
269745   18       1   6      107              0    12           3


In [34]:
 train_df, val_df = train_test_split( train_df, train_size=.95, random_state=99, shuffle=True )

print(train_df.info())
print(val_df.info())

print("train size: ", len(train_df))
print("valid size: ", len(val_df))

gc.collect()

print("Training...")

num_boost_round=MAX_ROUNDS
early_stopping_rounds=EARLY_STOP

xgtrain = lgb.Dataset(train_df[predictors].values, label=train_df[target].values,
                      feature_name=predictors,
                      categorical_feature=categorical
                      )
del train_df
gc.collect()

xgvalid = lgb.Dataset(val_df[predictors].values, label=val_df[target].values,
                      feature_name=predictors,
                      categorical_feature=categorical
                      )
del val_df
gc.collect()

evals_results = {}

bst = lgb.train(lgb_params, 
                 xgtrain, 
                 valid_sets=[xgtrain, xgvalid], 
                 valid_names=['train','valid'], 
                 evals_result=evals_results, 
                 num_boost_round=num_boost_round,
                 early_stopping_rounds=early_stopping_rounds,
                 verbose_eval=10, 
                 feval=None)

n_estimators = bst.best_iteration

print("\nModel Report")
print("n_estimators : ", n_estimators)
print(metrics+":", evals_results['valid'][metrics][n_estimators-1])

del xgvalid

<class 'pandas.core.frame.DataFrame'>
Int64Index: 643031 entries, 333285 to 686020
Data columns (total 7 columns):
app              643031 non-null uint16
device           643031 non-null uint16
os               643031 non-null uint16
channel          643031 non-null uint16
is_attributed    643031 non-null uint8
hour             643031 non-null uint8
in_test_hh       643031 non-null uint8
dtypes: uint16(4), uint8(3)
memory usage: 11.7 MB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 33844 entries, 436733 to 187041
Data columns (total 7 columns):
app              33844 non-null uint16
device           33844 non-null uint16
os               33844 non-null uint16
channel          33844 non-null uint16
is_attributed    33844 non-null uint8
hour             33844 non-null uint8
in_test_hh       33844 non-null uint8
dtypes: uint16(4), uint8(3)
memory usage: 628.0 KB
None
train size:  643031
valid size:  33844


/Users/aronak/miniconda3/envs/ml_3_6/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


484

Training...


81

81

/Users/aronak/miniconda3/envs/ml_3_6/lib/python3.5/site-packages/lightgbm/basic.py:1036: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/Users/aronak/miniconda3/envs/ml_3_6/lib/python3.5/site-packages/lightgbm/basic.py:681: UserWarning: categorical_feature in param dict is overrided.
  warnings.warn('categorical_feature in param dict is overrided.')


Training until validation scores don't improve for 50 rounds.
[10]	train's auc: 0.970178	valid's auc: 0.967905
[20]	train's auc: 0.971495	valid's auc: 0.970504
[30]	train's auc: 0.974873	valid's auc: 0.974847
[40]	train's auc: 0.977151	valid's auc: 0.978299
[50]	train's auc: 0.978551	valid's auc: 0.980056
[60]	train's auc: 0.98015	valid's auc: 0.98149
[70]	train's auc: 0.981723	valid's auc: 0.982113
[80]	train's auc: 0.982743	valid's auc: 0.983076
[90]	train's auc: 0.983598	valid's auc: 0.983148
[100]	train's auc: 0.984212	valid's auc: 0.984482
[110]	train's auc: 0.984871	valid's auc: 0.983569
[120]	train's auc: 0.985422	valid's auc: 0.98426
[130]	train's auc: 0.98584	valid's auc: 0.984353
[140]	train's auc: 0.986293	valid's auc: 0.983904
[150]	train's auc: 0.98664	valid's auc: 0.983338
[160]	train's auc: 0.986791	valid's auc: 0.983605
[170]	train's auc: 0.986844	valid's auc: 0.98362
Early stopping, best iteration is:
[126]	train's auc: 0.985651	valid's auc: 0.98498

Model Report
n_est